# explore tweet frequency, price reaction

In [45]:
##Loading in the data
import pandas as pd

# Load Musk tweets
tweets = pd.read_csv("../data/raw/elon_musk_tweets.csv", parse_dates=["date"])

# Load TSLA price data
prices = pd.read_csv("../data/raw/tsla_ohlcv_daily.csv", skiprows=2, index_col=0, parse_dates=True)
prices.index.name = "Date"
prices = prices.reset_index()
prices.columns = ["Date", "Close", "High", "Low", "Open", "Volume"]


# Quick look
print(tweets.head())
print(prices.head())



                    id  user_name user_location           user_description  \
0  1544379368478212100  Elon Musk           NaN  Mars & Cars, Chips & Dips   
1  1544377493263720450  Elon Musk           NaN  Mars & Cars, Chips & Dips   
2  1544377130590552064  Elon Musk           NaN  Mars & Cars, Chips & Dips   
3  1544375575724400645  Elon Musk           NaN  Mars & Cars, Chips & Dips   
4  1544375148605853699  Elon Musk           NaN  Mars & Cars, Chips & Dips   

                user_created  user_followers  user_friends  user_favourites  \
0  2009-06-02 20:12:29+00:00       101240855           115            13503   
1  2009-06-02 20:12:29+00:00       101240806           115            13503   
2  2009-06-02 20:12:29+00:00       101240806           115            13503   
3  2009-06-02 20:12:29+00:00       101240806           115            13503   
4  2009-06-02 20:12:29+00:00       101240806           115            13503   

   user_verified                      date  \
0         

In [46]:
tweets["date"] = pd.to_datetime(tweets["date"])
tweets["tweet_day"] = tweets["date"].dt.floor("d")

print(tweets["tweet_day"].head())

0   2022-07-05 00:00:00+00:00
1   2022-07-05 00:00:00+00:00
2   2022-07-05 00:00:00+00:00
3   2022-07-05 00:00:00+00:00
4   2022-07-05 00:00:00+00:00
Name: tweet_day, dtype: datetime64[ns, UTC]


In [47]:
# Add the next day close into the table so that we can compair current and next close
prices["next_close"] = prices["Close"].shift(-1)
print(prices.head())

        Date       Close        High         Low        Open     Volume  \
0 2022-07-01  227.263336  230.229996  222.119995  227.000000   74460300   
1 2022-07-05  233.066666  233.146667  216.166672  223.000000   84581100   
2 2022-07-06  231.733337  234.563339  227.186661  230.779999   71853600   
3 2022-07-07  244.543335  245.363327  232.210007  233.919998   81930600   
4 2022-07-08  250.763336  254.979996  241.160004  242.333328  101854200   

   next_close  
0  233.066666  
1  231.733337  
2  244.543335  
3  250.763336  
4  234.343338  


In [48]:
tweets["tweet_day"] = tweets["tweet_day"].dt.tz_localize(None)
merged = pd.merge(tweets, prices, left_on="tweet_day", right_on="Date", how="inner")
print(merged.head())

                    id  user_name user_location           user_description  \
0  1544379368478212100  Elon Musk           NaN  Mars & Cars, Chips & Dips   
1  1544377493263720450  Elon Musk           NaN  Mars & Cars, Chips & Dips   
2  1544377130590552064  Elon Musk           NaN  Mars & Cars, Chips & Dips   
3  1544375575724400645  Elon Musk           NaN  Mars & Cars, Chips & Dips   
4  1544375148605853699  Elon Musk           NaN  Mars & Cars, Chips & Dips   

                user_created  user_followers  user_friends  user_favourites  \
0  2009-06-02 20:12:29+00:00       101240855           115            13503   
1  2009-06-02 20:12:29+00:00       101240806           115            13503   
2  2009-06-02 20:12:29+00:00       101240806           115            13503   
3  2009-06-02 20:12:29+00:00       101240806           115            13503   
4  2009-06-02 20:12:29+00:00       101240806           115            13503   

   user_verified                      date  ... favorite

In [51]:
cleaned = merged[["date", "text", "tweet_day", "Date", "Open", "Close", "next_close"]].copy()
print(cleaned.head())

                       date  \
0 2022-07-05 17:55:09+00:00   
1 2022-07-05 17:47:42+00:00   
2 2022-07-05 17:46:15+00:00   
3 2022-07-05 17:40:05+00:00   
4 2022-07-05 17:38:23+00:00   

                                                text  tweet_day       Date  \
0  @BillyM2k I find the gold toe sock – inevitabl... 2022-07-05 2022-07-05   
1                 Sock Con, the conference for socks 2022-07-05 2022-07-05   
2  Always something new for the magazine cover an... 2022-07-05 2022-07-05   
3                   @ExplainThisBob This guy gets it 2022-07-05 2022-07-05   
4  Sock tech is so advanced that you can get pret... 2022-07-05 2022-07-05   

    Open       Close  next_close  
0  223.0  233.066666  231.733337  
1  223.0  233.066666  231.733337  
2  223.0  233.066666  231.733337  
3  223.0  233.066666  231.733337  
4  223.0  233.066666  231.733337  


In [53]:
#adding in the logic to have an index indicating if the price went up or down that day
cleaned["next_day_return"] = (cleaned["next_close"] - cleaned["Close"]) / cleaned["Close"]
cleaned["label"] = (cleaned["next_day_return"] > 0).astype(int)

print(cleaned.head())

                       date  \
0 2022-07-05 17:55:09+00:00   
1 2022-07-05 17:47:42+00:00   
2 2022-07-05 17:46:15+00:00   
3 2022-07-05 17:40:05+00:00   
4 2022-07-05 17:38:23+00:00   

                                                text  tweet_day       Date  \
0  @BillyM2k I find the gold toe sock – inevitabl... 2022-07-05 2022-07-05   
1                 Sock Con, the conference for socks 2022-07-05 2022-07-05   
2  Always something new for the magazine cover an... 2022-07-05 2022-07-05   
3                   @ExplainThisBob This guy gets it 2022-07-05 2022-07-05   
4  Sock tech is so advanced that you can get pret... 2022-07-05 2022-07-05   

    Open       Close  next_close  next_day_return  label  
0  223.0  233.066666  231.733337        -0.005721      0  
1  223.0  233.066666  231.733337        -0.005721      0  
2  223.0  233.066666  231.733337        -0.005721      0  
3  223.0  233.066666  231.733337        -0.005721      0  
4  223.0  233.066666  231.733337        -0.005721